In [1]:
import xgboost as xgb
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, roc_curve, precision_recall_curve
import pandas as pd
import sklearn.model_selection as msel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import seaborn as sns

In [42]:
df = pd.read_csv("heart_NO_Missing_Data.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,num
0,28,1,2,130.0,132.000000,0,2,185.0,0,0.0,0
1,29,1,2,120.0,243.000000,0,0,160.0,0,0.0,0
2,29,1,2,140.0,250.472119,0,0,170.0,0,0.0,0
3,30,0,1,170.0,237.000000,0,1,170.0,0,0.0,0
4,31,0,2,100.0,219.000000,0,1,150.0,0,0.0,0


In [43]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'num'],
      dtype='object')

In [44]:
# usunięcie features 
df = df.drop(['age', 'sex', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach'],axis=1)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cp       292 non-null    object 
 1   exang    292 non-null    object 
 2   oldpeak  292 non-null    float64
 3   num      292 non-null    object 
dtypes: float64(1), object(3)
memory usage: 9.2+ KB


In [55]:
df['cp'] = df['cp'].astype(object)
df['num'] = df['num'].astype(object)
df['exang'] = df['exang'].astype(object)
df['oldpeak'] = df['oldpeak'].astype(object)

In [56]:
encoder = LabelEncoder()
for col in ('cp', 'num', 'exang', 'oldpeak'):
    df['cp'] = encoder.fit_transform(df['cp'])
    df['num'] = encoder.fit_transform(df['num'])
    df['exang'] = encoder.fit_transform(df['exang'])
    df['oldpeak'] = encoder.fit_transform(df['oldpeak'])

MODELOWANIE

In [57]:
# PODZIAŁ NA FEATURES I TARGET
X = df.drop('num',axis=1)
y = df['num']

In [58]:
X.head()

,cp,exang,oldpeak
0,1,0,0
1,1,0,0
2,1,0,0
3,0,0,0
4,1,0,0


In [59]:
# PODZIAŁ NA ZBIÓR TESTOWY I TRENINGOWY
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

Tworzę model z trzema najważniejszymi features, używam parametrów z GridSearchCV

In [80]:
# TWORZENIE MODELU
xgb_cl = xgb_cl = xgb.XGBClassifier() 

# FITOWANIE
xgb_cl.fit(X_train, y_train)

# PREDYKCJA
preds = xgb_cl.predict(X_test)

EWALUACJA

In [81]:
accuracy_score(y_test, preds)

0.8409090909090909

In [ ]:
# na podstawie 3 features model w 84% przewiduje czy ktoś ma chorobę serca czy nie

In [82]:
# Ocena modelu
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.88      0.85      0.87        27
           1       0.78      0.82      0.80        17

    accuracy                           0.84        44
   macro avg       0.83      0.84      0.83        44
weighted avg       0.84      0.84      0.84        44



Wyniki weighted i macro avg oraz accuracy są bardzo podobne co oznacza że model dobrze działa.

In [83]:
confusion_matrix(y_test, preds)

array([[23,  4],
       [ 3, 14]])

In [84]:
roc_auc_score (y_test, preds)

0.8376906318082789

In [85]:
xgb_cl = xgb.XGBClassifier()

In [86]:
# Tuningowanie hiperparametrów
param_grid = {
    "max_depth": [3, 4, 5, 7, 9, 15],
    "learning_rate": [0.1, 0.01, 0.05, 0.2, 0.5, 0.8],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    'n_estimators': [1, 4, 5, 40, 60, 80, 100],
}

In [87]:
from sklearn.model_selection import GridSearchCV

# Init klasyfikator
xgb_cl = xgb.XGBClassifier(objective="reg:logistic")

# Init Grid Search
grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

# Fit
_ = grid_cv.fit(X_train, y_train)


In [88]:
grid_cv.best_score_

0.8910813111962538

In [89]:
grid_cv.best_params_

{'gamma': 0.25,
 'learning_rate': 0.5,
 'max_depth': 4,
 'n_estimators': 40,
 'reg_lambda': 10,
 'scale_pos_weight': 3}

In [90]:
grid_cv.best_estimator_.feature_importances_

array([0.27032492, 0.593118  , 0.13655709], dtype=float32)

In [91]:
feat_import = grid_cv.best_estimator_.feature_importances_
imp_feats = pd.DataFrame(index=X.columns,data=feat_import,columns=['Importance'])
imp_feats

,Importance
cp,0.270325
exang,0.593118
oldpeak,0.136557


In [ ]:
# OSTATECZNY MODEL

In [72]:
final_cl = xgb.XGBClassifier(
    objective="reg:logistic",
    **grid_cv.best_params_,
    colsample_bytree=0.5,
    subsample=0.8
)

final_cl.fit(X, y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.25, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=40, n_jobs=None, num_parallel_tree=None,
              objective='reg:logistic', predictor=None, ...)

In [57]:
import pickle
# ZAPISANIE MODELU
with open('final_model.pkl', 'wb') as file:
    pickle.dump(final_cl, file)

In [58]:
# ZAŁADOWANIE MODELU Z PLIKU
with open('final_model.pkl', 'rb') as file:
    final_cl = pickle.load(file)

In [67]:
# PREDYKCJE NA NOWYCH DANYCH
cardiac_score = np.array([[2, 0, 0.0]]) # _cp, _exang, _oldpeak scores
predictions = final_cl.predict(cardiac_score)
print(predictions)

[0]
